# California Season Prediction

## Imports and Functions

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor


from sklearn.svm import SVR
import os
import numpy as np
from xgboost import XGBRegressor
import sys

In [2]:
# Add the folder to the Python path

os.chdir("../")
# change working directory to project's root path
print(os.getcwd())

FIRST_YEAR= 1972

c:\Users\marti\Desktop\data\hw_extra


In [3]:
folder_path = os.path.abspath("functions/") #INPUT_PATH)#'path_to_your_folder')  # Replace with the actual folder path
sys.path.insert(0, folder_path)

from Predictions import (
    PredictionExperiment,
    PredictionModel
)

In [4]:
metadata = pd.read_csv("data/new_features/california/metadata.csv")
metadata = metadata[metadata["frequency"]=="2monthly"]
metadata

,id,filename,boxes,top_n,var_thresh,modes,frequency,season,methods
0,9bf745db,predictor_9bf745db_1.parquet,0,5,NaN,3,2monthly,1,ps
1,9bf745db,predictor_9bf745db_2.parquet,0,5,NaN,3,2monthly,2,ps
2,9bf745db,predictor_9bf745db_3.parquet,0,5,NaN,3,2monthly,3,ps
3,9bf745db,predictor_9bf745db_4.parquet,0,5,NaN,3,2monthly,4,ps
4,9bf745db,predictor_9bf745db_5.parquet,0,5,NaN,3,2monthly,5,ps
...,...,...,...,...,...,...,...,...,...
715,02d3904a,predictor_02d3904a_8.parquet,0,30,0.15,3,2monthly,8,ps
716,02d3904a,predictor_02d3904a_9.parquet,0,30,0.15,3,2monthly,9,ps
717,02d3904a,predictor_02d3904a_10.parquet,0,30,0.15,3,2monthly,10,ps
718,02d3904a,predictor_02d3904a_11.parquet,0,30,0.15,3,2monthly,11,ps


In [5]:
kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=1)
regressors = [LinearRegression(),RandomForestRegressor(random_state=42, n_estimators=5), RandomForestRegressor(random_state=42, n_estimators=10), SVR(kernel='rbf'),
                XGBRegressor(random_state=42, n_estimators=10, learning_rate=0.1),XGBRegressor(random_state=42, n_estimators=15, learning_rate=0.1), GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10)]
name_regressors = ["Linear", "RF5", "RF10", "SVR-rbf", "XGB10", "XGB15", "GPR"]
indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]

In [6]:
id_experiments = metadata["id"].unique()
for id in id_experiments:
    data = {i: pd.read_parquet(f"data/new_features/california/predictor_{id}_{i}.parquet") for i in range(1,13)}

In [7]:
experiment_1 = PredictionExperiment(data, indices_of_interest, regressors, name_regressors, 5)
experiment_1.execute_experiment()

In [8]:
experiment_1.get_metrics("r2", "prediction", thresh=0.5)

,Model,Season,Metric,Stage,HWN,HWF,HWD,HWM,HWA,Average
0,Linear,1,r2,prediction,0.72,0.57,0.52,0.23,0.27,0.46
1,RF5,1,r2,prediction,0.79,0.45,0.27,0.23,0.30,0.41
2,RF10,1,r2,prediction,0.78,0.52,0.34,0.23,0.35,0.45
3,SVR-rbf,1,r2,prediction,0.60,0.27,0.24,0.54,0.51,0.43
4,XGB10,1,r2,prediction,0.54,0.25,0.08,0.20,0.20,0.25
5,XGB15,1,r2,prediction,0.68,0.33,0.17,0.26,0.27,0.34
6,GPR,1,r2,prediction,0.51,0.17,0.14,0.25,0.22,0.26
7,Linear,2,r2,prediction,-3.06,-3.81,-2.38,-2.82,-2.66,-2.95
8,RF5,2,r2,prediction,-1.60,-1.69,-0.55,-0.30,-0.94,-1.01
9,RF10,2,r2,prediction,-1.87,-1.00,-0.53,-0.60,-0.93,-0.99


In [9]:
experiment_1.top_results("r2", 5)

,Model,Season,Index,Value
0,RF10,7,HWD,0.838435
1,RF5,7,HWD,0.804394
2,RF5,1,HWN,0.791447
3,RF10,1,HWN,0.779331
4,RF5,10,HWF,0.761483
